In [7]:
# Reference : https://towardsdatascience.com/master-machine-learning-decision-trees-from-scratch-with-python-de75b0494bcd


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from collections import Counter
import pandas as pd


class RootNode:
    '''
    Helper class which implements a single tree node.
    '''
    def __init__(self, feature=None, threshold=None, data_left=None, data_right=None, info_gain=None, value=None):
        self.attr = feature
        self.threshold = threshold
        self.left_data = left_data
        self.right_data = right_data
        self.gain = info_gain
        self.value = value

class DecisionTree:
    '''
    Class which implements a decision tree classifier algorithm.
    '''
    def __init__(self, min_samples_split=2, max_depth=9):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None
        
    @staticmethod
    def _entropy(s):
        '''
        Helper function, calculates entropy from an array of integer values.
        
        :param s: list
        :return: float, entropy value
        '''
        # Convert to integers to avoid runtime errors
        counts = np.bincount(np.array(s, dtype=np.int64))
        # Probabilities of each class label
        percentages = counts / len(s)

        # Caclulate entropy
        entropy = 0
        for pct in percentages:
            if pct > 0:
                entropy += pct * np.log2(pct)
        return -entropy
    
    def _information_gain(self, parent, left_child, right_child):
        '''
        Helper function, calculates information gain from a parent and two child nodes.
        
        :param parent: list, the parent node
        :param left_child: list, left child of a parent
        :param right_child: list, right child of a parent
        :return: float, information gain
        '''
        num_left = len(left_child) / len(parent)
        num_right = len(right_child) / len(parent)
        
        # One-liner which implements the previously discussed formula
        return self._entropy(parent) - (num_left * self._entropy(left_child) + num_right * self._entropy(right_child))
    
    def _best_split(self, X, y):
        '''
        Helper function, calculates the best split for given features and target
        
        :param X: np.array, features
        :param y: np.array or list, target
        :return: dict
        '''
        best_split = {}
        best_info_gain = -1
        n_rows, n_cols = X.shape
        
        # For every dataset feature
        for f_idx in range(n_cols):
            X_curr = X[:, f_idx]
            # For every unique value of that feature
            for threshold in np.unique(X_curr):
                # Construct a dataset and split it to the left and right parts
                # Left part includes records lower or equal to the threshold
                # Right part includes records higher than the threshold
                df = np.concatenate((X, y.reshape(1, -1).T), axis=1)
                df_left = np.array([row for row in df if row[f_idx] <= threshold])
                df_right = np.array([row for row in df if row[f_idx] > threshold])

                # Do the calculation only if there's data in both subsets
                if len(df_left) > 0 and len(df_right) > 0:
                    # Obtain the value of the target variable for subsets
                    y = df[:, -1]
                    y_left = df_left[:, -1]
                    y_right = df_right[:, -1]

                    # Caclulate the information gain and save the split parameters
                    # if the current split if better then the previous best
                    gain = self._information_gain(y, y_left, y_right)
                    if gain > best_info_gain:
                        best_split = {
                            'feature_index': f_idx,
                            'threshold': threshold,
                            'df_left': df_left,
                            'df_right': df_right,
                            'gain': gain
                        }
                        best_info_gain = gain
        return best_split
    
    def _build(self, X, y, depth=0):
        '''
        Helper recursive function, used to build a decision tree from the input data.
        
        :param X: np.array, features
        :param y: np.array or list, target
        :param depth: current depth of a tree, used as a stopping criteria
        :return: Node
        '''
        n_rows, n_cols = X.shape
        # Check to see if a node should be leaf node
        if n_rows >= self.min_samples_split and depth <= self.max_depth:
            # Get the best split
            best = self._best_split(X, y)
            # If the split isn't pure
            if best['gain'] > 0:
                # Build a tree on the left
                left = self._build(
                    X=best['df_left'][:, :-1], 
                    y=best['df_left'][:, -1], 
                    depth=depth + 1
                )
                right = self._build(
                    X=best['df_right'][:, :-1], 
                    y=best['df_right'][:, -1], 
                    depth=depth + 1
                )
                return Node(
                    feature=best['feature_index'], 
                    threshold=best['threshold'], 
                    data_left=left, 
                    data_right=right, 
                    gain=best['gain']
                )
        # Leaf node - value is the most common target value 
        return Node(
            value=Counter(y).most_common(1)[0][0]
        )
    
    def fit(self, X, y):
        '''
        Function used to train a decision tree classifier model.
        
        :param X: np.array, features
        :param y: np.array or list, target
        :return: None
        '''
        # Call a recursive function to build the tree
        self.root = self._build(X, y)
        
    def _predict(self, x, tree):
        '''
        Helper recursive function, used to predict a single instance (tree traversal).
        
        :param x: single observation
        :param tree: built tree
        :return: float, predicted class
        '''
        # Leaf node
        if tree.value != None:
            return tree.value
        attr_val = x[tree.feature]
        
        # Go to the left
        if attr_val <= tree.threshold:
            return self._predict(x=x, tree=tree.data_left)
        
        # Go to the right
        if attr_val > tree.threshold:
            return self._predict(x=x, tree=tree.data_right)
        
    def predict(self, X):
        '''
        Function used to classify new instances.
        
        :param X: np.array, features
        :return: np.array, predicted classes
        '''
        # Call the _predict() function for every observation
        return [self._predict(x, self.root) for x in X]
    
    def print_dt(self, tree, position, depth = 0):
        if(depth ==0):
            print("Iterate through decision tree, printing out values.")
            tree = self.root
        if(depth < 3):
            print("At depth = "+ str(depth)+" i.e., "+ position)
            print("feature is(column) "+str(tree.feature))
            print("threshold = "+ tree.threshold)
            print("gain = "+ str(tree.gain))
            self.print_dt(tree.data_left, "Left Node", depth + 1)
            self.print_dt(tree.data_right,"Right Node", depth + 1)
            


df = pd.read_csv('/content/tic-tac-toe_train.csv', header=None)
df_data = np.array(df)
train_data_x = df_data[:, :-1]
y = df_data[:, -1]
train_data_y = np.where(y == 'win', 1, 0)

df_test = pd.read_csv('/content/tic-tac-toe_test.csv', header=None)
df_data_test = np.array(df_test)
test_data_x = df_data_test[:, :-1]
test_data_y = df_data_test[:, -1]
y_test = np.where(y_test_data == 'win', 1, 0)

for i in range(1,10):
    model = DecisionTree(min_samples_split=2, max_depth=i)
    clf = model.fit(train_data_x, y_train)
    predict_train = model.predict(train_data_x)
    print("Accuracy for Depth " +str(i)+" on train data is")
    print(accuracy_score(y_train,predict_train))
    
    predict_test = model.predict(test_data_x)
    print("Accuracy for Depth " +str(i)+" on test data is")
    print(accuracy_score(y_test,predict_test))
print("From the Accuracy values over fitting occurs for Depth = 7,8,9")

Accuracy for Depth 1 on train data is
0.732
Accuracy for Depth 1 on test data is
0.66
Accuracy for Depth 2 on train data is
0.78
Accuracy for Depth 2 on test data is
0.75
Accuracy for Depth 3 on train data is
0.848
Accuracy for Depth 3 on test data is
0.78
Accuracy for Depth 4 on train data is
0.88
Accuracy for Depth 4 on test data is
0.78
Accuracy for Depth 5 on train data is
0.968
Accuracy for Depth 5 on test data is
0.84
Accuracy for Depth 6 on train data is
0.984
Accuracy for Depth 6 on test data is
0.87
Accuracy for Depth 7 on train data is
0.992
Accuracy for Depth 7 on test data is
0.86
Accuracy for Depth 8 on train data is
1.0
Accuracy for Depth 8 on test data is
0.85
Accuracy for Depth 9 on train data is
1.0
Accuracy for Depth 9 on test data is
0.85
From the Accuracy values over fitting occurs for Depth = 7,8,9
